In [1]:
import json
import re
import sys
import os
import ast

from tqdm import tqdm
from collections import Counter
from pathlib import Path

In [2]:
import sys
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
import nltk
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from gensim.models.fasttext import FastText
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt

from gensim.models import KeyedVectors

from nltk.corpus import stopwords
import gensim.downloader as api
import string
from sklearn.semi_supervised import LabelSpreading, LabelPropagation

/home/g/g01107/.local/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# **`Clean and Process Data `**

In [3]:
import nltk
nltk.download("stopwords")
nltk.download('vader_lexicon')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/g/g01107/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/g/g01107/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
df = pd.read_json("./Trec_data/org_combined_labeled.json", orient='records',lines=True)
df

,eventID,eventType,postID,postCategories,postPriority,postText
0,stormJorge2020,typhoon,1231307896362807298,[Irrelevant],Low,Flood Warning: River Severn at Hanley Castle a...
1,stormJorge2020,typhoon,1231569665043976192,[Irrelevant],Low,Flood Warning: River Ouse at Naburn Lock 12:46...
2,stormJorge2020,typhoon,1232264304067477504,[Irrelevant],Low,Our Assistant Director of Care and Support kin...
3,stormJorge2020,typhoon,1232070602778959872,[Irrelevant],Low,@hollywills please can you help support @HopeR...
4,stormJorge2020,typhoon,1232648900105965568,[Irrelevant],Low,Police order 'immediate evacuation' in Shropsh...
...,...,...,...,...,...,...
91510,whaleyBridgeCollapse2020,flood,1155430270457323520,[Irrelevant],Low,Flood Alert: River Ecclesbourne in Derbyshire ...
91511,whaleyBridgeCollapse2020,flood,1156993824591417346,"[Location, EmergingThreats, MultimediaShare, N...",High,Dam at Whaley Bridge in Peak District threaten...
91512,whaleyBridgeCollapse2020,flood,1157020257388769280,"[ThirdPartyObservation, Location, MultimediaSh...",Low,Floods in Whaley Bridge today.\nhttps://t.co/7...
91513,whaleyBridgeCollapse2020,flood,1156926115069485056,"[MovePeople, ThirdPartyObservation, Location, ...",Critical,Evacuation of Whaley Bridge | Derbyshire Const...


In [ ]:
#Remove certain non-neccessary columns
cols = ['eventID',
 'eventType',
 'postID',
 'postCategories',
 'postPriority',
 'postText']
df = df[cols]
df

In [5]:
stop_words = stopwords.words('english')
stop_words.extend(["http", "https", "rt", "@", ":", "t.co", "co", "amp", "&amp;", "...", "\n", "\r"])
stop_words.extend(string.punctuation)

In [6]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = str(sent)
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True)
        yield(sent)


In [7]:
# Look into what shape this ./labeled.json
df = pd.read_json("./Trec_data/labeled.json", orient='records', lines=True)
data = df.postText.values.tolist()
data_words = list(sent_to_words(data))

In [8]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [9]:
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_trf', disable=['parser', 'ner'])
    for sent in tqdm(texts): #Code to see that it is progressing
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]
    return texts_out

In [10]:
data_ready = process_words(data_words)
df['processed_text']=data_ready

100%|██████████| 91515/91515 [2:23:38<00:00, 10.62it/s]  


In [16]:
df.to_json('./Trec_data/PR_all_Labeled.json', orient='records', lines=True)
df

,eventID,eventType,postID,postCategories,postPriority,postText,processed_text
0,stormJorge2020,typhoon,1231307896362807296,[Irrelevant],Low,Flood Warning: River Severn at Hanley Castle a...,"[flood, february]"
1,stormJorge2020,typhoon,1231569665043976192,[Irrelevant],Low,Flood Warning: River Ouse at Naburn Lock 12:46...,"[flood, naburn_lock, february]"
2,stormJorge2020,typhoon,1232264304067477504,[Irrelevant],Low,Our Assistant Director of Care and Support kin...,"[assistant, director, care, support, kindly, l..."
3,stormJorge2020,typhoon,1232070602778959872,[Irrelevant],Low,@hollywills please can you help support @HopeR...,"[help, support, following, recent, flooding, c..."
4,stormJorge2020,typhoon,1232648900105965568,[Irrelevant],Low,Police order 'immediate evacuation' in Shropsh...,"[police, order, flooding, send]"
...,...,...,...,...,...,...,...
91510,whaleyBridgeCollapse2020,flood,1155430270457323520,[Irrelevant],Low,Flood Alert: River Ecclesbourne in Derbyshire ...,"[flood, alert, river]"
91511,whaleyBridgeCollapse2020,flood,1156993824591417344,"[Location, EmergingThreats, MultimediaShare, N...",High,Dam at Whaley Bridge in Peak District threaten...,"[peak, district, threaten, burst, gofh, pb, nc..."
91512,whaleyBridgeCollapse2020,flood,1157020257388769280,"[ThirdPartyObservation, Location, MultimediaSh...",Low,Floods in Whaley Bridge today.\nhttps://t.co/7...,"[flood, today]"
91513,whaleyBridgeCollapse2020,flood,1156926115069485056,"[MovePeople, ThirdPartyObservation, Location, ...",Critical,Evacuation of Whaley Bridge | Derbyshire Const...,"[evacuation, constabulary]"


# Generating Similarity Scores and Matrix
### **Mean and Cosine Similarity(each event with all other event-types)**


In [13]:
import pickle
#import logging
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler, Normalizer
from sklearn.pipeline import make_pipeline
from scipy import sparse
import numpy as np
import os.path
import re
from tqdm import tqdm
import warnings
import pandas as pd

In [14]:
data = pd.aread_json("./Trec_data/PR_all_Labeled.json", orient='records',lines=True)

In [30]:
#Removes rows that have no processed _text (Dropped a third)
data['l'] = data.apply(lambda row: len(row['processed_text']), axis=1)
data= data.query("l >1") 
data.drop(columns=['l'], inplace=True)
data

,eventID,eventType,postID,postCategories,postPriority,postText,processed_text,ir
602,tennesseeDerecho2020,storm,1257045884560461824,"[Location, MultimediaShare, ContextualInformat...",Low,Nashville Mayor Floats 32 Percent Property Tax...,"[mayor, float, percent, property, tax, hike, c...",0
604,tennesseeDerecho2020,storm,1256433237301137408,"[Weather, Location]",Low,Summerdale Bartlett TN Weather 11:00 PM Temp: ...,"[tn, weather, pm, bar, change, rain, today, mo...",0
605,tennesseeDerecho2020,storm,1256913902224334848,"[Weather, Location, Discussion]",Low,"@tanueki TN Air 001, Mac tower.\nWind 60 degre...","[tn, air, mac, tower, wind, degree, knot, runw...",0
606,tennesseeDerecho2020,storm,1257044059593588736,"[Weather, Location, MultimediaShare, Hashtags,...",High,#STORM12ALERT SEVERE THUNDERSTORM WARNING for ...,"[storm, alert, warning, county, tn, pm, wd]",0
607,tennesseeDerecho2020,storm,1257058413298712576,"[Weather, Location, MultimediaShare, Official,...",High,Nashville metro find your Safe Spots now. We a...,"[metro, find, safe, spot, roughly, minute, awa...",0
...,...,...,...,...,...,...,...,...
91509,whaleyBridgeCollapse2020,flood,1156949866402189312,"[Location, EmergingThreats, MultimediaShare, F...",High,Oh no #WhaleyBridge has collapsed (minor). Be ...,"[collapse, minor, safe]",0
91511,whaleyBridgeCollapse2020,flood,1156993824591417344,"[Location, EmergingThreats, MultimediaShare, N...",High,Dam at Whaley Bridge in Peak District threaten...,"[peak, district, threaten, burst, gofh, pb, nc...",0
91512,whaleyBridgeCollapse2020,flood,1157020257388769280,"[ThirdPartyObservation, Location, MultimediaSh...",Low,Floods in Whaley Bridge today.\nhttps://t.co/7...,"[flood, today]",0
91513,whaleyBridgeCollapse2020,flood,1156926115069485056,"[MovePeople, ThirdPartyObservation, Location, ...",Critical,Evacuation of Whaley Bridge | Derbyshire Const...,"[evacuation, constabulary]",0


In [42]:
# generate sentence embedding
class SBERT:

    def __init__(self, lang="en"):
        from sentence_transformers import SentenceTransformer
        self.name = "SBERT"
        if lang == "fr":
            self.model = SentenceTransformer(
                "/home/bmazoyer/Dev/pytorch_bert/output/sts_fr_long_multilingual_bert-2019-10-01_15-07-03")
        elif lang == "en": #Does this need to be changed?
            self.model = SentenceTransformer(
                # "bert-large-nli-stsb-mean-tokens"
                "roberta-large-nli-stsb-mean-tokens"
            )
# roberta-large-nli-stsb-mean-tokens
    def compute_vectors(self, data):
        data["postText"] = data.postText.str.slice(0, 500)
        vectors = np.array(self.model.encode(data.postText.tolist()))
        return vectors

In [43]:
sbert=SBERT()

In [45]:
v=sbert.compute_vectors(data)
data['sbert_emb']=[item for item in v]

In [ ]:
#Save SBERT data
data.to_json('./Trec_data/SBERT_Labelled.json', orient='records', lines=True)
data

In [ ]:
data = pd.read_json("./Trec_data/SBERT_Labelled.json", orient='records',lines=True)

In [67]:
# remove irrelevant tweets
#data.loc[(data.eventid == 'parisAttacks2015'),'event_type']='shooting' #Do we need?
def label_ir_tweets(postCategories):

    if 'Irrelevant' in postCategories:
        return 1
    else:
        return 0
data['ir'] = [label_ir_tweets(x) for x in data['postCategories']]
data=data.query("ir == 0")
data.drop(columns=['ir'], inplace=True)
#data=data.query("label ==1")

In [68]:
data['postCategories'].explode().unique()

array(['Factoid', 'MultimediaShare', nan, 'MovePeople',
       'FirstPartyObservation', 'Discussion', 'News', 'Official',
       'Sentiment', 'EmergingThreats', 'ServiceAvailable',
       'ThirdPartyObservation', 'Donations', 'Advice', 'NewSubEvent',
       'Weather', 'CleanUp', 'Volunteer', 'Hashtags', 'InformationWanted',
       'OriginalEvent', 'ContextualInformation', 'SearchAndRescue',
       'GoodsServices', 'Location'], dtype=object)

In [69]:
data['eventType'].unique()

array(['wildfire', 'earthquake', 'flood', 'typhoon', 'shooting',
       'bombing', 'covid', 'explosion', 'storm', 'hostage', 'tornado',
       'fire'], dtype=object)

In [70]:
from numpy import dot
from numpy.linalg import norm

In [71]:
#Can't handle the list of postType
def generate_similarity_matrix (frame, grouping, group_types):
    #generate similarity scores dataframe
    group_ranks=pd.DataFrame()
    for heldout_event in group_types:

        training = frame[frame[grouping] != heldout_event]
        test = frame[frame[grouping] == heldout_event]

        ref=np.mean(test["sbert_emb"], axis=0)

        grpups=training.groupby(grouping) #Might need to be changed 
        ranks={}
        ranks["reference-group"]=heldout_event
        for name, group in grpups:
           val=np.mean(group["sbert_emb"], axis=0)
           cos_sim = dot(ref, val)/(norm(ref)*norm(val))
           ranks[name]=cos_sim

        # event_ranks[heldout_event]=ranks
        #print(ranks)
        group_ranks = group_ranks.append(ranks, ignore_index=True)
    group_ranks.set_index("reference-group",inplace=True)
    #group_ranks=frame.groupby(grouping) #Does the label==1 need to change?
    return group_ranks


# Prep for Visualization

Actual visualization on Presentation_visuals notebook

In [72]:
import seaborn as sns

In [73]:
#is this even required? and is this based off the 4 different sources?
events=[ 
'2014_Philippines_Typhoon_Hagupi',
 '2015_Cyclone_Pam',
 'albertaFloods2013',
 'albertaWildfires2019',
 'australiaBushfire2013',
 'cycloneKenneth2019',
 'fireYMM2016',
 'hurricaneFlorence2018',
 'keralaFloods2019',
 'manilaFloods2013',
 'philipinnesFloods2012',
 'queenslandFloods2013',
 'southAfricaFloods2019',
 'typhoonHagupit2014',
 'typhoonYolanda2013'
]
event_types=['typhoon', 'storm', 'wildfire', 'covid', 'flood',
       'shooting', 'earthquake', 'explosion', 'hostage', 'fire',
       'tornado', 'bombing']
event_types

['typhoon',
 'storm',
 'wildfire',
 'covid',
 'flood',
 'shooting',
 'earthquake',
 'explosion',
 'hostage',
 'fire',
 'tornado',
 'bombing']

In [74]:
event_ranks = generate_similarity_matrix(data, 'eventType', event_types)
event_ranks = event_ranks.replace(np.nan, 1)
event_ranks.to_csv("./Trec_data/event_ranks.csv")
event_ranks

,bombing,covid,earthquake,explosion,fire,flood,hostage,shooting,storm,tornado,wildfire,typhoon
reference-group,,,,,,,,,,,,
typhoon,0.699150,0.733873,0.819881,0.620435,0.654905,0.888971,0.523911,0.475814,0.844313,0.699773,0.783521,1.000000
storm,0.571836,0.651923,0.627162,0.611186,0.572461,0.735063,0.476997,0.504595,1.000000,0.766818,0.733257,0.844313
wildfire,0.658392,0.686796,0.676957,0.627472,0.710281,0.771168,0.473985,0.531951,0.733257,0.723049,1.000000,0.783521
covid,0.613739,1.000000,0.719096,0.589605,0.595344,0.669775,0.566324,0.623835,0.651923,0.638793,0.686796,0.733873
flood,0.631070,0.669775,0.759224,0.589725,0.681597,1.000000,0.472704,0.475011,0.735063,0.627928,0.771168,0.888971
shooting,0.642831,0.623835,0.469381,0.594687,0.481176,0.475011,0.639686,1.000000,0.504595,0.547695,0.531951,0.475814
earthquake,0.682907,0.719096,1.000000,0.604083,0.573381,0.759224,0.434615,0.469381,0.627162,0.645873,0.676957,0.819881
explosion,0.617208,0.589605,0.604083,1.000000,0.682002,0.589725,0.444642,0.594687,0.611186,0.745527,0.627472,0.620435
hostage,0.549946,0.566324,0.434615,0.444642,0.467154,0.472704,1.000000,0.639686,0.476997,0.474765,0.473985,0.523911


In [75]:
critical_types=['Low', 'Medium', 'High', 'Critical']

In [76]:
critical_ranks = generate_similarity_matrix(data, 'postPriority', critical_types) #check what this variable is called in the dataframe
critical_ranks = critical_ranks.replace(np.nan, 1)
critical_ranks.to_csv("./Trec_data/critical_ranks.csv")
critical_ranks

,Critical,High,Medium,Unknown,Low
reference-group,,,,,
Low,0.914040,0.962109,0.972243,0.425645,1.000000
Medium,0.935539,0.984607,1.000000,0.448886,0.972243
High,0.949960,1.000000,0.984607,0.451244,0.962109
Critical,1.000000,0.949960,0.935539,0.471790,0.914040


In [77]:
info = data[['postCategories', 'sbert_emb']]
info

,postCategories,sbert_emb
0,[Factoid],"[0.7833897, 0.4475655, -0.6625698, 0.60922027,..."
1,[MultimediaShare],"[0.010711178, -0.19763231, -0.34852412, 0.8226..."
2,[MultimediaShare],"[-0.74097896, 0.9875894, -0.042316787, 0.17517..."
3,[],"[0.23670046, 1.1524885, 0.022244314, 0.0050623..."
4,[MultimediaShare],"[-0.81128556, 1.0994077, 0.62657094, 0.4951987..."
...,...,...
72467,"[ThirdPartyObservation, Location, NewSubEvent,...","[-0.0197716, 0.6021651, -0.12217062, 0.2699645..."
72468,"[FirstPartyObservation, MultimediaShare, Advice]","[-0.9820343, -0.98338723, -0.6184144, -0.35399..."
72470,"[ThirdPartyObservation, Location, EmergingThre...","[0.35711387, -0.88471466, -0.051269475, 0.8173..."
72471,"[ThirdPartyObservation, Location, MultimediaSh...","[0.97791696, -0.16704077, -0.644486, -0.230477..."


In [80]:
info_types = ['Location', 'MultimediaShare', 'ContextualInformation', 'Weather', 'Discussion', 'Hashtags', 'News', 'Official', 'EmergingThreats', 'FirstPartyObservation', 'Factoid', 'ThirdPartyObservation', 'MovePeople', 'Sentiment', 'Advice', 'GoodsServices', 'Donations', 'ServiceAvailable', 'SearchAndRescue', 'NewSubEvent', 'Volunteer', 'CleanUp', 'InformationWanted', 'OriginalEvent']



for j in tqdm(info_types):
    filename = j
    outfile = open(filename,'wb')
    r = -1
    df = pd.DataFrame()

    for i in info['postCategories']:
        r += 1
        if j in i:
            df = df.append(info.iloc[r])
            df['postCategories'] = j
    pickle.dump(df,outfile)
    outfile.close()

100%|██████████| 24/24 [07:06<00:00, 17.76s/it]


In [81]:
#1
infile = open('Location','rb')
Location = pickle.load(infile)
infile.close()
#2
infile = open('MultimediaShare','rb')
MultimediaShare = pickle.load(infile)
infile.close()
#3
infile = open('ContextualInformation','rb')
ContextualInformation = pickle.load(infile)
infile.close()
#4
infile = open('Weather','rb')
Weather = pickle.load(infile)
infile.close()
#5
infile = open('Discussion','rb')
Discussion = pickle.load(infile)
infile.close()
#6
infile = open('Hashtags','rb')
Hashtags = pickle.load(infile)
infile.close()
#7
infile = open('News','rb')
News = pickle.load(infile)
infile.close()
#8
infile = open('Official','rb')
Official = pickle.load(infile)
infile.close()
#9
infile = open('EmergingThreats','rb')
EmergingThreats = pickle.load(infile)
infile.close()
#10
infile = open('FirstPartyObservation','rb')
FirstPartyObservation = pickle.load(infile)
infile.close()
#11
infile = open('Factoid','rb')
Factoid = pickle.load(infile)
infile.close()
#12
infile = open('ThirdPartyObservation','rb')
ThirdPartyObservation = pickle.load(infile)
infile.close()
#13
infile = open('MovePeople','rb')
MovePeople = pickle.load(infile)
infile.close()
#14
infile = open('Sentiment','rb')
Sentiment = pickle.load(infile)
infile.close()
#15
infile = open('Advice','rb')
Advice = pickle.load(infile)
infile.close()
#16
infile = open('GoodsServices','rb')
GoodsServices = pickle.load(infile)
infile.close()
#17
infile = open('Donations','rb')
Donations = pickle.load(infile)
infile.close()
#18
infile = open('ServiceAvailable','rb')
ServiceAvailable = pickle.load(infile)
infile.close()
#19
infile = open('SearchAndRescue','rb')
SearchAndRescue = pickle.load(infile)
infile.close()
#20
infile = open('NewSubEvent','rb')
NewSubEvent = pickle.load(infile)
infile.close()
#21
infile = open('Volunteer','rb')
Volunteer = pickle.load(infile)
infile.close()
#22
infile = open('CleanUp','rb')
CleanUp = pickle.load(infile)
infile.close()
#23
infile = open('InformationWanted','rb')
InformationWanted = pickle.load(infile)
infile.close()
#24
infile = open('OriginalEvent','rb')
OriginalEvent = pickle.load(infile)
infile.close()



In [82]:
dataframes = [Location, MultimediaShare, ContextualInformation, Weather, Discussion, Hashtags, News, Official, EmergingThreats, FirstPartyObservation, Factoid, ThirdPartyObservation, MovePeople, Sentiment, Advice, GoodsServices, Donations, ServiceAvailable, SearchAndRescue, NewSubEvent, Volunteer, CleanUp, InformationWanted, OriginalEvent]

frame = pd.DataFrame(columns = ['postCategories', 'sbert_emb'])

for i in tqdm(dataframes):
    frame = frame.append(i)
    
frame

100%|██████████| 24/24 [00:00<00:00, 92.25it/s] 


,postCategories,sbert_emb
19043,Location,"[-0.31157446, 0.88926494, -0.7680027, -0.73744..."
19044,Location,"[0.9682974, 0.18655212, 0.9547924, 0.68871427,..."
19063,Location,"[0.32405418, -0.09517649, 0.4217278, -0.334986..."
19075,Location,"[0.013062832, 0.26928332, 0.35960522, 0.395124..."
19112,Location,"[0.6689324, 1.0096077, 0.43295857, 0.6388316, ..."
...,...,...
65372,OriginalEvent,"[1.0163454, 0.451552, -0.0434816, -0.5077652, ..."
65373,OriginalEvent,"[0.3175971, -0.22463116, -0.18630771, 0.739010..."
65374,OriginalEvent,"[0.24771717, 0.31029183, 0.601508, -0.49695462..."
65375,OriginalEvent,"[1.3043524, 0.6517899, 0.6881786, -0.5156773, ..."


In [83]:
filename = 'info_ranks'
outfile = open(filename,'wb')
pickle.dump(frame,outfile)
outfile.close()

In [84]:
info_ranks = generate_similarity_matrix(frame, 'postCategories', info_types) #check what this variable is called in the dataframe
info_ranks = info_ranks.replace(np.nan, 1)
info_ranks.to_csv("./Trec_data/info_ranks.csv")
info_ranks

,Advice,CleanUp,ContextualInformation,Discussion,Donations,EmergingThreats,Factoid,FirstPartyObservation,GoodsServices,Hashtags,...,News,Official,OriginalEvent,SearchAndRescue,Sentiment,ServiceAvailable,ThirdPartyObservation,Volunteer,Weather,Location
reference-group,,,,,,,,,,,,,,,,,,,,,
Location,0.910273,0.926771,0.909490,0.939677,0.640251,0.968561,0.976492,0.910733,0.775853,0.973110,...,0.993436,0.964875,0.946812,0.815233,0.836592,0.821855,0.990650,0.710033,0.937182,1.000000
MultimediaShare,0.906807,0.926039,0.919587,0.948742,0.659103,0.956475,0.979221,0.922940,0.783377,0.973867,...,0.996017,0.961533,0.954527,0.829478,0.848786,0.834652,0.993406,0.722388,0.931147,0.996018
ContextualInformation,0.878101,0.833615,1.000000,0.981943,0.641716,0.817910,0.881418,0.859214,0.747825,0.894239,...,0.913575,0.901228,0.905120,0.786723,0.819647,0.832272,0.923975,0.716994,0.779302,0.909490
Weather,0.858801,0.886802,0.779302,0.840668,0.593643,0.956185,0.920643,0.883130,0.718466,0.931307,...,0.934713,0.920603,0.849597,0.747922,0.779809,0.751924,0.908478,0.652824,1.000000,0.937182
Discussion,0.915335,0.869118,0.981943,1.000000,0.674186,0.869597,0.914438,0.917191,0.788132,0.941814,...,0.943273,0.929150,0.923691,0.832823,0.876307,0.853784,0.947818,0.744881,0.840668,0.939677
Hashtags,0.952289,0.927088,0.894239,0.941814,0.710975,0.951594,0.948006,0.961326,0.833650,1.000000,...,0.974933,0.967602,0.933396,0.859119,0.909723,0.863126,0.959492,0.782692,0.931307,0.973110
News,0.911239,0.927644,0.913575,0.943273,0.658174,0.959834,0.982759,0.926006,0.787438,0.974933,...,1.000000,0.971129,0.952946,0.831175,0.850082,0.838048,0.991179,0.724998,0.934713,0.993436
Official,0.950290,0.907163,0.901228,0.929150,0.657936,0.942141,0.935189,0.924574,0.784397,0.967602,...,0.971129,1.000000,0.903540,0.811879,0.837349,0.868487,0.952444,0.744523,0.920603,0.964875
EmergingThreats,0.895868,0.900748,0.817910,0.869597,0.589804,1.000000,0.940405,0.886252,0.745897,0.951594,...,0.959834,0.942141,0.892546,0.780639,0.806583,0.760512,0.942350,0.662119,0.956185,0.968561
